In [14]:
import pdfplumber
import json
import re
from elasticsearch import Elasticsearch


Config ignore table when extract_text

In [22]:
def curves_to_edges(cs):
    edges = []
    for c in cs:
        edges += pdfplumber.utils.rect_to_edges(c)
    return edges

def not_within_bboxes(obj, bboxes):
    def obj_in_bbox(_bbox):
        v_mid = (obj["top"] + obj["bottom"]) / 2
        h_mid = (obj["x0"] + obj["x1"]) / 2
        x0, top, x1, bottom = _bbox
        return (h_mid >= x0) and (h_mid < x1) and (v_mid >= top) and (v_mid < bottom)
    return not any(obj_in_bbox(__bbox) for __bbox in bboxes)
  
def get_table_config(page):
  config = {
      "vertical_strategy": "explicit",
      "horizontal_strategy": "explicit",
      "explicit_vertical_lines": curves_to_edges(page.curves + page.edges),
      "explicit_horizontal_lines": curves_to_edges(page.curves + page.edges),
      "intersection_y_tolerance": 10,
  }
  return config

Create raw data after read from pdf (ignore table)

In [23]:

with pdfplumber.open('s3-userguide.pdf') as pdf:
    # init raw data array
    lines_data = []
    # loop each page with 
    for page_num, page in enumerate(pdf.pages, start=1):
        lines = page.extract_text(layout=True).split('\n')
        for line_num, line in enumerate(lines, start=1):
            lines_data.append({
                "page": page_num,
                "line": line_num,
                "content": line,
                "title": lines[2],
                "section_title": lines[-4]
            })

    with open('raw.json', 'w', encoding='utf-8') as f:
        json.dump(lines_data, f, ensure_ascii=False, indent=4)

Clean, add rule and group raw data to data which can import to ES

In [24]:
with open('raw.json', 'r', encoding='utf-8') as f:
    lines_data = json.load(f)

filtered_lines_data = []
current_content = ''
for line_data in lines_data:
    content = line_data['content'].strip()
    if content:
        content = re.sub(r'\s+', ' ', content)
        if "API Version 2006-03-01" not in content and "Amazon Simple Storage Service" not in content:
            if (current_content != '' and current_content[-1] != '.') or content[0].islower() or content[0] == '-' or content[0] == '•':
              if content[0] == '-' or content[0] == '•':
                current_content += '\n' +  content
              else:        
                current_content += ' ' + content
            else:
                if current_content:
                    line_data['title'] = re.sub(r'\s+', ' ', line_data['title'].strip())
                    line_data['content'] = current_content
                    line_data['section_title'] = line_data['section_title'].strip().split("API Version")[0].strip()
                    if line_data['section_title'] and "Table of Contents" not in line_data['content']:
                        filtered_lines_data.append(line_data)
                current_content = content
if current_content:
    line_data['content'] = current_content
    filtered_lines_data.append(line_data)

with open('output.json', 'w', encoding='utf-8') as f:
    json.dump(filtered_lines_data, f, ensure_ascii=False, indent=4)

Put data to ES

In [ ]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'scheme': 'http'}])

with open('output.json', 'r') as file:
    data = json.load(file)

for doc in data:
    es.index(index='s3_guide', body=doc)

/var/folders/yz/tvpwz_9d6c56fms92tltys800000gn/T/ipykernel_55600/2731284087.py:8: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.index(index='s3_guide', body=doc)
